## Reading Time Prediction from Syntactic Surprisal

Main codebase of final project for Prof. Frank's LING 380 course

Written by Mandy Osuji, Rishika Veeramachaneni, and Andrew Perun

Requires CSV files relating to Filler.csv and ClassicGardenPathSet.csv, as processed by LING_380_Preprocess.ipynb, to be stored at paths data & data_GP

For data availability, train the regressions using syntactic surprisals produced by all four of Arehalli's LMs, but separate/compare the predicted reading times by model.

Note **minimal garbage collection takes place**; RAM bottlenecks must be assessed before running subsequent code.


In [1]:
import pandas as pd
from tqdm import tqdm
import os
import re
import numpy as np
import gc
import psutil
import itertools

from scipy.stats import fligner, ks_2samp
import matplotlib.pyplot as plt
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from joblib import dump, load

import patsy

In [3]:
# set your base file system, data will be written in made subdirectories

base = "/Users/andrewperun/Desktop/LING_380/Final_Proj/Trial2/"

gp_cases_base = base + "single_GP"
os.makedirs(gp_cases_base, exist_ok=True)

prediction_base = base + "predictions"
os.makedirs(prediction_base, exist_ok=True)

exclude_base = base + "exclude_GP"
os.makedirs(exclude_base, exist_ok=True)

model_base = base + "regressions"
os.makedirs(model_base, exist_ok=True)

#### Function Declarations: Data Preprocessing

To add lagged values, log frequencies/count, sentence position, and produce minimal dataframes for regression training.

In [21]:
def apply_with_progress(grouped, func):
    """
    Applies a function to each group in a grouped DataFrame with progress tracking and memory optimization.

    Parameters:
    -----------
    grouped : DataFrameGroupBy
        The grouped DataFrame to process.
    func : callable
        Function to apply to each group.

    Returns:
    --------
    DataFrame
        Concatenated result of applying the function to all groups.
    """
    results = []

    for _, group in tqdm(grouped, desc="Processing groups"):
        results.append(func(group))

    print("Length of results: " + str(len(results)))

    for i, df in enumerate(results):
        if df.empty:
            print("Empty DataFrame at index " + str(i)) # ensure all dfs nonempty

    print("Concatenating into chunks...")
    chunk_size = 10000
    concatenated_chunks = []

    for i in tqdm(range(0, len(results), chunk_size), desc="Concatenating", unit="chunk"): # Chunk and concatenate
        current_chunk = results[i:i + chunk_size]
        if current_chunk:  # if chunk is not empty
            # Only append non-empty DataFrames to concatenated_chunks
            non_empty_chunks = [df for df in current_chunk if not df.empty]
            if non_empty_chunks:
                concatenated_chunks.append(pd.concat(non_empty_chunks))
            results[i:i + chunk_size] = [None] * len(current_chunk) # 'safe' memory management (haha)

    print("Length of concatenated chunks: " + str(len(concatenated_chunks)))
    for i in range(len(concatenated_chunks)):
        print(concatenated_chunks[i].head())

    print(f"Memory usage before concat: {psutil.virtual_memory().used / 1e6:.2f} MB") # memory update 1
    result_df = pd.concat(concatenated_chunks, ignore_index=True)
    print(f"Memory usage after concat: {psutil.virtual_memory().used / 1e6:.2f} MB")
    gc.collect()  # this could be re-factored & pushed earlier, but it works

    return result_df

def add_lags(group):
    """
    Adds lagged columns (1, 2, 3 steps back) for lexical, syntactic, frequency, and length predictors to a group.

    Parameters:
    -----------
    group : DataFrame
        A single group DataFrame.

    Returns:
    --------
    DataFrame
        The group with new lagged columns added.
    """
    return group.assign(
        lex_surprisal_p1_s=group['lex_surprisal_s'].shift(1),
        lex_surprisal_p2_s=group['lex_surprisal_s'].shift(2),
        lex_surprisal_p3_s=group['lex_surprisal_s'].shift(3),
        syn_surprisal_p1_s=group['syn_surprisal_s'].shift(1),
        syn_surprisal_p2_s=group['syn_surprisal_s'].shift(2),
        syn_surprisal_p3_s=group['syn_surprisal_s'].shift(3),
        logfreq_p1_s=group['logfreq_s'].shift(1),
        logfreq_p2_s=group['logfreq_s'].shift(2),
        logfreq_p3_s=group['logfreq_s'].shift(3),
        length_p1_s=group['length_s'].shift(1),
        length_p2_s=group['length_s'].shift(2),
        length_p3_s=group['length_s'].shift(3)
    )

In [22]:
def process_data(data_path, output_path, base_path):
    """
    Processes a dataset by filtering, applying lagged predictors, and saving the cleaned output.

    Steps:
        Load and Filter Data: Read data from data_path and ensure no missing values in critical columns.
        Apply Lags: Add lagged predictors for grouped data and optionally save the lagged dataset.
        Process and Clean: Add sentence length, drop rows with missing predictors, and select essential columns.
        Save Final Data: Write the cleaned dataset to output_path.

    Parameters:
    -----------
    data_path : str
        Path to the input dataset CSV file.
    output_path : str
        Path to save the processed dataset.
    save_lags : bool, optional
        If True, saves the dataset with lagged predictors to `lags_path`. NOTE: This may cause excessive memory usage.
    lags_path : str, optional
        Path to save the lagged dataset if `save_lags=True`.

    Returns:
    --------
    None
        Saves the processed data to the specified output path.
    """
    merged = pd.read_csv(data_path, header=0)
    #merged.head()
    filtered_rows = merged[merged['logfreq_s'].isna() | merged['lex_surprisal_s'].isna() | merged['syn_surprisal_s'].isna() | merged['length_s'].isna() ]
    if len(filtered_rows) > 0:
        raise ValueError("Dataset has rows with missing values.")
    
    merged_withlags = apply_with_progress(
        merged.groupby(['item', 'MD5', 'model']),
        add_lags
    )
    
    merged_withlags.to_csv(os.path.join(base, "with_lags.csv"))

    merged_withlags['sent_length'] = merged_withlags['Sentence'].str.split(" ").apply(len)
    
    columns_to_check = [
        'lex_surprisal_s', 'lex_surprisal_p1_s', 'lex_surprisal_p2_s', 'lex_surprisal_p3_s',
        'syn_surprisal_s', 'syn_surprisal_p1_s', 'syn_surprisal_p2_s', 'syn_surprisal_p3_s',
        'logfreq_s', 'logfreq_p1_s', 'logfreq_p2_s', 'logfreq_p3_s'
    ]

    dropped = merged_withlags.dropna(subset=columns_to_check)

    columns_to_keep = [
        "RT", "syn_surprisal_s", "syn_surprisal_p1_s", "syn_surprisal_p2_s",
        "lex_surprisal_s", "lex_surprisal_p1_s", "lex_surprisal_p2_s",
        "WordPosition", "logfreq_s", "logfreq_p1_s", "logfreq_p2_s",
        "length_s", "length_p1_s", "length_p2_s", "MD5", "item", "model"
    ]
    dropped_minimal = dropped[columns_to_keep]
    print("Data writen out to " + output_path) 
    dropped_minimal.to_csv(output_path, index=False)


## Processing filler & GP surprisal data:

In [4]:
data = base + "fmerged_mod.csv"
out = base + "fdropped_minimal.csv"

In [ ]:
process_data(data, out, base)

In [39]:
data_GP = base + "merged_mod.csv"
out_GP = base + "dropped_minimal.csv"

In [ ]:
process_data(data_GP, out_GP, base)

## Statistical Testing:

Comparing syntactic surprisal and reading times between regression-trained (only filler) and predictive (garden path case) data

In [23]:
def test_KS_FK(set1, set2):
    # KS test
    stat_KS, p_value_KS = ks_2samp(set1, set2)
    print("K-S Statistic:", stat_KS)
    print("P-value:", p_value_KS)
    if p_value_KS < 0.05:
        print("Distributions significantly different.")
    else:
        print("Distributions not significantly different.")

    # FK test
    stat_FK, p_value_FK = fligner(set1, set2)

    # Print results
    print("Fligner-Killeen Statistic:", stat_FK)
    print("P-value:", p_value_FK)
    
    # Interpretation
    if p_value_FK < 0.05:
        print("Variances significantly different.")
    else:
        print("Variances not significantly different.")

##### Testing Syntactic Surprisal of Filler and All GP Cases

In [6]:
with_lags = pd.read_csv(os.path.join(base, "with_lags.csv"), header=0)
fdropped = pd.read_csv(out_GP, header=0)

In [163]:
synsurp_GP = list(with_lags["syn_surprisal_s"])
synsurp_F  = list(fdropped["syn_surprisal_s"])

In [164]:
test_KS_FK(synsurp_GP, synsurp_F)

K-S Statistic: 0.07450928404435464
P-value: 0.0
Distributions significantly different.
Fligner-Killeen Statistic: 11009.802752211754
P-value: 0.0
Variances significantly different.


##### Testing Syntactic Surprisal of Filler and Unambiguous GP Cases

In [165]:
synsurp_Unam_GP = list(with_lags[with_lags['Type'].str.endswith('_UAMB', na=False)]["syn_surprisal_s"])

In [166]:
test_KS_FK(synsurp_Unam_GP, synsurp_F)

K-S Statistic: 0.06634687648868465
P-value: 0.0
Distributions significantly different.
Fligner-Killeen Statistic: 22101.98849543789
P-value: 0.0
Variances significantly different.


##### Testing Syntactic Surprisal of ambiguous GP cases against each other

In [7]:
synsurp_am_MVRR = list(with_lags[with_lags['Type'].str.endswith('MVRR_AMB', na=False)]["syn_surprisal_s"])
synsurp_am_NPS  = list(with_lags[with_lags['Type'].str.endswith('NPS_AMB', na=False)]["syn_surprisal_s"])
synsurp_am_NPZ =  list(with_lags[with_lags['Type'].str.endswith('NPZ_AMB', na=False)]["syn_surprisal_s"])

In [11]:
elems = [synsurp_am_MVRR, synsurp_am_NPS, synsurp_am_NPZ]
for i in list(itertools.combinations({0, 1, 2}, 2)):
    print(f"Testing {i[0]} and {i[1]}")
    test_KS_FK(elems[i[0]], elems[i[1]])

Testing 0 and 1
K-S Statistic: 0.05195633602969618
P-value: 0.0
Distributions significantly different.
Fligner-Killeen Statistic: 942.4621828964345
P-value: 5.7714612682572204e-207
Variances significantly different.
Testing 0 and 2
K-S Statistic: 0.09781969997807372
P-value: 0.0
Distributions significantly different.
Fligner-Killeen Statistic: 13534.869266336262
P-value: 0.0
Variances significantly different.
Testing 1 and 2
K-S Statistic: 0.10507654229120222
P-value: 0.0
Distributions significantly different.
Fligner-Killeen Statistic: 22489.51095001312
P-value: 0.0
Variances significantly different.


## Experiment 1:

Training regressions using syntactic surprisal on filler reading times + {2 GP cases}, to produce reading times for the third GP case.

In [24]:
def get_fillerGP(mindata_filler, mindata_GP, output_path_base): # use the outputs from process_data
    """
    Filters and combines datasets based on specific case exclusions and saves the results.

    Steps:
        1. Load Data: Reads the minimal filler dataset and the GP dataset.
        2. Filter GP Data: Retains rows in the GP dataset for specific 'Type' cases ('_AMB' and '_UAMB' for MVRR, NPS, NPZ).
        3. Exclude Cases: Iterates over the main cases (MVRR, NPS, NPZ), excluding one case at a time.

    Parameters:
    -----------
    mindata_filler : str
        Path to the CSV file containing the filler dataset.
    mindata_GP : str
        Path to the CSV file containing the GP dataset.
    output_path_base : str
        Directory path to save the resulting combined datasets.

    Returns:
    --------
    None
        Saves the combined datasets with excluded cases to the specified directory.
    """
    
    fdropped_min = pd.read_csv(mindata_filler, header=0)
    dropped_min = pd.read_csv(mindata_GP, header=0)

    all_cases = list(set(list(dropped_min['Type'])))
    gpath_cases = ['MVRR', 'NPS', 'NPZ']
    gpath_cases_full = [c + '_AMB' for c in gpath_cases] + [c + '_UAMB' for c in gpath_cases];
    
    dropped_min = dropped_min[dropped_min['Type'].isin([c + '_AMB' for c in gpath_cases] + [c + '_UAMB' for c in gpath_cases])]
    
    for exclude_case in gpath_cases:
        used_cases = [case for case in all_cases if case not in [exclude_case + '_AMB', exclude_case + '_UAMB']]
    
        filtered_rows = dropped_min[dropped_min['Type'].isin(used_cases)] # extract rows with those with cases in in_cases
        df_combined = pd.concat([fdropped_min, filtered_rows], ignore_index=True, join='outer')
        output_file = os.path.join(output_path_base, f"minimal_-{exclude_case}.csv")
        print("Data writen out to " + output_file)
        df_combined.to_csv(output_file, index=False)


In [25]:
def fit_regression(model_string, random_effect_string, dataset_path, model_out_path, optimizer = "bfgs"):

    dataset = pd.read_csv(dataset_path, header = 0)

    # Check consistent typing of all relevant columns
    cols = dataset.columns[:15]
    for col in cols:
        column_types = dataset[col].apply(type)
        if (column_types.nunique() != 1):
            print("Column " + str(col) + " inconsistent.") 
    
    filler_model_syn = smf.mixedlm(
        model_string,
        data=dataset,
        groups=dataset["item"],  
        re_formula=random_effect_string
    ).fit(method=optimizer, maxiter=100000, disp=True) 

    dump(filler_model_syn, model_out_path)

In [26]:
def predictRT(model_string, model_path, dataset_path):
    regression = load(model_path)
    new_data = pd.read_csv(dataset_path, header=0)

    fe_matrix = patsy.dmatrix( # retrieve fixed effects matrix using new_data
        model_string.split("~")[1], new_data, return_type="dataframe"
    )
    fe_params = regression.fe_params # predicting fixed effects
    new_data["fe_prediction"] = fe_matrix @ fe_params

    rand_effects = regression.random_effects # creating random effect matrix, filling na with 0
    new_data["rand_effects"] = new_data["item"].map(rand_effects).fillna(0)
    
    new_data["predicted_rt_syn"] = new_data["fe_prediction"] + new_data["rand_effects"] # get final RT predictions

    return new_data

In [27]:
def predictRT_dataset(model_string, model_path, dataset, output_path):
    regression = load(model_path)

    fe_matrix = patsy.dmatrix( # retrieve fixed effects matrix using new_data
        model_string.split("~")[1], dataset, return_type="dataframe"
    )
    fe_params = regression.fe_params # predicting fixed effects
    dataset["fe_prediction"] = fe_matrix @ fe_params

    rand_effects = regression.random_effects # creating random effect matrix, filling na with 0
    dataset["rand_effects"] = dataset["item"].map(rand_effects).fillna(0)
    
    dataset["predicted_rt_syn"] = dataset["fe_prediction"] + dataset["rand_effects"] # get final RT predictions

    dataset.to_csv(output_path, header=0)

##### Getting Datasets

In [ ]:
get_fillerGP(out, out_GP, exclude_base)

##### Fitting Models

In [17]:
syn_model = """
    RT ~ syn_surprisal_s + syn_surprisal_p1_s +
         syn_surprisal_p2_s + scale(WordPosition) +
         logfreq_s * length_s + logfreq_p1_s * length_p1_s +
         logfreq_p2_s * length_p2_s
    """

random_effect = "1 + syn_surprisal_s + syn_surprisal_p1_s + syn_surprisal_p2_s"

In [ ]:
fit_regression(syn_model, random_effect, 
               os.path.join(exclude_base, "minimal_-MVRR.csv"), 
               os.path.join(model_base, "model-MVRR.joblib"))

In [ ]:
fit_regression(syn_model, random_effect, 
               os.path.join(exclude_base, "minimal_-NPS.csv"), 
               os.path.join(model_base, "model-NPS.joblib"))

In [ ]:
fit_regression(syn_model, random_effect, 
               os.path.join(exclude_base, "minimal_-NPZ.csv"), 
               os.path.join(model_base, "model-NPZ.joblib"))

##### Predicting Reading Times

In [46]:
dropped_min = pd.read_csv(out_GP, header=0)

In [ ]:
dropped_min_only_MVRR = dropped_min[dropped_min['Type'].isin(['MVRR_UAMB', 'MVRR_AMB'])]
predictRT_dataset(syn_model, 
          os.path.join(model_base, "model-MVRR.joblib"), 
          dropped_min_only_MVRR,
          os.path.join(prediction_base, "MVRR_pred.csv"))

In [ ]:
dropped_min_only_NPS = dropped_min[dropped_min['Type'].isin(['NPS_UAMB', 'NPS_AMB'])]
predictRT_dataset(syn_model, 
          os.path.join(model_base, "model-NPS.joblib"), 
          dropped_min_only_NPS,
          os.path.join(prediction_base, "NPS_pred.csv"))

In [ ]:
dropped_min_only_NPZ = dropped_min[dropped_min['Type'].isin(['NPZ_UAMB', 'NPZ_AMB'])]
predictRT_dataset(syn_model, 
          os.path.join(model_base, "model-NPZ.joblib"), 
          dropped_min_only_NPZ,
          os.path.join(prediction_base, "NPZ_pred.csv"))

##### Inspecting Regressions

In [6]:
def inspect_model(model_path):
    print(f"Model at {model_path}:")
    regression = load(model_path)
    coeff = regression.params
    #pval = filler_model_syn.pvalues
    print("Coefficients:\n", coeff)
    #print("P-values:\n", pval)

In [7]:
inspect_model(os.path.join(model_base, "model-MVRR.joblib"))
inspect_model(os.path.join(model_base, "model-NPS.joblib"))
inspect_model(os.path.join(model_base, "model-NPZ.joblib"))

Model at /Users/andrewperun/Desktop/LING_380/Final_Proj/Trial2/regressions/model-MVRR.joblib:
Coefficients:
 Intercept                                      393.397933
syn_surprisal_s                                 11.229539
syn_surprisal_p1_s                               4.796593
syn_surprisal_p2_s                              -2.083444
scale(WordPosition)                             -0.690775
logfreq_s                                      -19.568904
length_s                                         8.743645
logfreq_s:length_s                              -5.257586
logfreq_p1_s                                   -16.766479
length_p1_s                                      3.687154
logfreq_p1_s:length_p1_s                        -3.889810
logfreq_p2_s                                    -5.265493
length_p2_s                                     -0.529202
logfreq_p2_s:length_p2_s                        -0.926852
Group Var                                        0.023498
Group x syn_surprisal

## Computing GP Effect


In [28]:
def split_predictions(preds_path):

    header = [
        "idx", "RT", "Construction", "syn_surprisal_s", "syn_surprisal_p1_s", "syn_surprisal_p2_s",
        "lex_surprisal_s", "lex_surprisal_p1_s", "lex_surprisal_p2_s",
        "WordPosition", "logfreq_s", "logfreq_p1_s", "logfreq_p2_s",
        "length_s", "length_p1_s", "length_p2_s", "MD5", "item", "model", "pred_RT", "eff1", "eff2"
    ]
    
    df = pd.read_csv(preds_path)
    
    items = df.iloc[:, -5].unique()  

    separated_preds = {}

    for value in items:
        subset_df = df[df.iloc[:, -5] == value]
        subset_df.columns = header
        
        amb_df = subset_df[subset_df['Construction'].str.endswith('_AMB', na=False)]
        uamb_df = subset_df[subset_df['Construction'].str.endswith('_UAMB', na=False)]

        # split by model as well
        amb_split = {model: amb_df[amb_df['model'] == model] for model in amb_df['model'].unique()}
        uamb_split = {model: uamb_df[uamb_df['model'] == model] for model in uamb_df['model'].unique()}


        # Add to the dictionary
        separated_preds[value] = {
            'all': subset_df,
            'amb': amb_df,
            'uamb': uamb_df,
            'amb_split': amb_split,
            'uamb_split': uamb_split,
        }

    return separated_preds

In [29]:
def dropna_nestedDict(d, columns):
    for key, value in d.items():
        if isinstance(value, dict):
            #print(f"Dropping in dict[{key}]")
            dropna_nestedDict(value, columns)
        elif isinstance(value, pd.DataFrame):
            d[key] = value.dropna(subset=columns)

In [30]:
def split_Lags(lags_path, columns_to_check):

    df = pd.read_csv(lags_path, header=0)
    
    items = df['item_x'].unique()  

    separated_preds = {}

    for value in items:
        subset_df = df[df['item_x'] == value]
        
        mvrr = subset_df[subset_df['Type'].str.startswith('MVRR', na=False)]
        nps = subset_df[subset_df['Type'].str.startswith('NPS', na=False)]
        npz = subset_df[subset_df['Type'].str.startswith('NPZ', na=False)]
        
        # split by uamb/amb and model
        mvrr_split = {
            construction: {
                model: mvrr[(mvrr['Type'] == construction) & (mvrr['model'] == model)]
                for model in ['m0', 'm1', 'm2', 'm3']
            }
            for construction in mvrr['Type'].unique()
        }

        nps_split = {
            construction: {
                model: nps[(nps['Type'] == construction) & (nps['model'] == model)]
                for model in ['m0', 'm1', 'm2', 'm3']
            }
            for construction in nps['Type'].unique()
        }

        npz_split = {
            construction: {
                model: npz[(npz['Type'] == construction) & (npz['model'] == model)]
                for model in ['m0', 'm1', 'm2', 'm3']
            }
            for construction in npz['Type'].unique()
        }
    
        # Add to the dictionary
        separated_preds[value] = {
            'mvrr': mvrr_split,
            'nps': nps_split,
            'npz': npz_split
        }
    
    # Traverse all dictionaries and drop na values
    dropna_nestedDict(separated_preds, columns_to_check)
    
    return separated_preds

In [31]:
def get_GPeffect_byModel(type_preds, lagged_truth, const):

    # type_preds is a prediction dataframe
    # lagged truth is lagged_sep
    # const is one of ['mvrr', 'nps', 'npz']
    
    ##### Get human garden path effect #####
    human_GP_effect = []
    for i in lagged_truth.keys(): # for each sentence
        unamb = lagged_truth[i][const][const.upper() + '_UAMB']['m0']
        amb =   lagged_truth[i][const][const.upper() + '_AMB']['m0']

        effect_list = []

        # for each position around disambiguating word
        for j in [-1, 0, 1, 2]:
            df1 = unamb[unamb['ROI'] == j]['RT']
            df2 = amb[amb['ROI'] == j]['RT']
            min_len = min([len(df1), len(df2)])
            
            df1 = df1.reset_index(drop=True).iloc[:min_len]
            df2 = df2.reset_index(drop=True).iloc[:min_len]

            # get garden path effect at that position for that sentence, and append to list
            diffs = (df2-df1).tolist()
            effect_list.append(np.mean(diffs))

        # append this sentences' GP effects
        human_GP_effect.append(effect_list)

    # now average GP effects through all sentences
    human_GP_effect = zip(*human_GP_effect)
    human_GP_effect = [sum(pos) / len(pos) for pos in human_GP_effect]


    ##### Get Predicted path effect #####
    GP_effects = []
    for i in type_preds.keys(): # for each sentence
        
        i_list = []
        
        # get word position of disambiguating word
        truth = lagged_truth[i][const][const.upper() + '_UAMB']['m0']
        ambig_disambig = truth['disambPositionAmb'].iloc[0]
        unambig_disambig = truth['disambPositionUnamb'].iloc[0]
        #print(f"Item {i}: {ambig_disambig}")
        #print(f"Item {i}: {unambig_disambig}")

        
        for model in ['m0', 'm1', 'm2', 'm3']: # for each model
            
            m_list = []
            
            unamb = type_preds[i]['uamb_split'][model]
            amb = type_preds[i]['amb_split'][model]
            
            for j in [-1, 0, 1, 2]: # for each position around disambiguating word
                df1 = unamb[unamb['WordPosition'] == j + unambig_disambig]['pred_RT']
                df2 = amb[amb['WordPosition'] == j + ambig_disambig]['pred_RT']
                min_len = min([len(df1), len(df2)])
            
                df1 = df1.reset_index(drop=True).iloc[:min_len]
                df2 = df2.reset_index(drop=True).iloc[:min_len]

                diffs = (df2-df1).tolist()
                m_list.append(np.mean(diffs))

            i_list.append(m_list)
            
        GP_effects.append(i_list)


    trans = list(zip(*GP_effects))
    means = []
    for group in trans:
        mean_sublist = np.mean(group, axis=0)
        means.append(mean_sublist)

    model_GP_effects = [list(sublist) for sublist in means]
    
    return human_GP_effect, model_GP_effects
    
    

##### Getting GP effect from new RT predictions

In [102]:
MVRR_preds = split_predictions(os.path.join(prediction_base, "MVRR_pred.csv"))
NPS_preds  = split_predictions(os.path.join(prediction_base, "NPS_pred.csv"))
NPZ_preds  = split_predictions(os.path.join(prediction_base, "NPZ_pred.csv"))

In [189]:
columns_to_check = [
        'lex_surprisal_s', 'lex_surprisal_p1_s', 'lex_surprisal_p2_s', 'lex_surprisal_p3_s',
        'syn_surprisal_s', 'syn_surprisal_p1_s', 'syn_surprisal_p2_s', 'syn_surprisal_p3_s',
        'logfreq_s', 'logfreq_p1_s', 'logfreq_p2_s', 'logfreq_p3_s']

lagged_sep = split_Lags(os.path.join(base, "with_lags.csv"), columns_to_check)

In [255]:
MVRR_human_GP, MVRR_model_GPs = get_GPeffect_byModel(MVRR_preds, lagged_sep, 'mvrr')
NPS_human_GP,  NPS_model_GPs  = get_GPeffect_byModel(NPS_preds,  lagged_sep, 'nps')
NPZ_human_GP,  NPZ_model_GPs  = get_GPeffect_byModel(NPZ_preds,  lagged_sep, 'npz')
gpath_cases = ['MVRR', 'NPS', 'NPZ']

##### Inspecting and saving human & model GP effects

In [257]:
print(MVRR_human_GP)
print(NPS_human_GP)
print(NPZ_human_GP)

[-7.5711756860257005, 41.737653392294625, 167.4356757427677, 94.73673452609182]
[17.100729397502587, 32.337040211112644, 49.61063704069727, 18.32235934651217]
[5.757101983650362, 84.22053326940727, 119.79536401912738, 51.919479000115764]


In [11]:
data = {
    "MVRR_human_GP": MVRR_human_GP,
    "NPS_human_GP": NPS_human_GP,
    "NPZ_human_GP": NPZ_human_GP
}
df = pd.DataFrame(data)
path = os.path.join(base, "Human_GPs.csv")
df.to_csv(path, index=False)

In [16]:
print(MVRR_model_GPs)
print(NPS_model_GPs)
print(NPZ_model_GPs)
full_list = [MVRR_model_GPs, NPS_model_GPs, NPZ_model_GPs]
gpath_cases = ['MVRR', 'NPS', 'NPZ']

[[-3.230610490613884, 17.186624745226453, 13.909817099294337, 2.2180659127159674], [-1.977681477480095, 12.984437937021061, 10.920510773908903, 1.7143983885677025], [-0.3013085566932446, 14.892478155166147, 10.116550408035563, 0.9995437859549909], [-5.242110117622686, 21.60587020400983, 17.71627788178112, 1.8141812661563808]]
[[6.256116414395542, 11.630508235821075, 11.708929578695923, 3.170007797366162], [5.277581204845733, 12.583516500227669, 14.053373960941236, 4.0867581164792774], [5.244316622120549, 14.200459442989859, 11.36316805921044, 1.2870135924841382], [5.6489282516340396, 9.430140166559898, 10.486839725228455, 1.9440673295129438]]
[[3.2046302403903617, 16.51320009171937, 14.815773993312334, 4.195241024517017], [3.0223932015042894, 14.058265704299508, 14.128443585210062, 2.955786262760578], [3.330855016117338, 15.306694710857982, 11.44057917507238, 1.2919196282660672], [4.657642411593202, 15.83749954078255, 15.382879095511548, 2.663166419090245]]


In [15]:
for i in range(len(full_list)):
    data = {
        "m0": full_list[i][0],
        "m1": full_list[i][1],
        "m2": full_list[i][2],
        "m3": full_list[i][3]
    }
    df = pd.DataFrame(data)
    title = gpath_cases[i] + "_GPs.csv"
    path = os.path.join(base, title)
    df.to_csv(path, index=False)
    

## Experiment 2:

Training regressions on only 2 Garden Path cases, then used to produce reading times for the third GP case.

##### Creating new GP only, training regression, predicting RTs

In [14]:
dropped_min = pd.read_csv(out_GP, header=0)
gpath_cases = ['MVRR', 'NPS', 'NPZ']

In [ ]:
mvrr = [c for c in gpath_cases if c != 'MVRR']
dropped_min_MVRR = dropped_min[dropped_min['Type'].isin([c + '_AMB' for c in mvrr] + [c + '_UAMB' for c in mvrr])]

filler_model_syn = smf.mixedlm(
    syn_model,
    data=dropped_min_MVRR,
    groups=dropped_min_MVRR["item"],  
    re_formula="1 + syn_surprisal_s + syn_surprisal_p1_s + syn_surprisal_p2_s"
).fit(method="lbfgs", maxiter=100000, disp=True) 

dump(filler_model_syn, model_base + "model-MVRR_onlyGP.joblib")


In [ ]:
dropped_min_only_MVRR = dropped_min[dropped_min['Type'].isin(['MVRR_UAMB', 'MVRR_AMB'])]

predictRT_dataset(syn_model, 
                  os.path.join(model_base, "model-MVRR_onlyGP.joblib"),
                  dropped_min_only_MVRR, 
                  os.path.join(prediction_base, "MVRR_pred_onlyGP.csv"))


In [ ]:
nps = [c for c in gpath_cases if c != 'NPS']
dropped_min_NPS = dropped_min[dropped_min['Type'].isin([c + '_AMB' for c in nps] + [c + '_UAMB' for c in nps])]

filler_model_syn = smf.mixedlm(
    syntactic_model,
    data=dropped_min_NPS,
    groups=dropped_min_NPS["item"],  
    re_formula="1 + syn_surprisal_s + syn_surprisal_p1_s + syn_surprisal_p2_s"
).fit(method="lbfgs", maxiter=100000, disp=True) 

dump(filler_model_syn, model_base + "model-NPS_onlyGP.joblib")

In [ ]:
dropped_min_only_NPS = dropped_min[dropped_min['Type'].isin(['NPS_UAMB', 'NPS_AMB'])]


predictRT_dataset(syn_model, 
                  os.path.join(model_base, "model-NPS_onlyGP.joblib"),
                  dropped_min_only_NPS, 
                  os.path.join(prediction_base, "NPS_pred_onlyGP.csv"))

In [ ]:
npz = [c for c in gpath_cases if c != 'NPZ']
dropped_min_NPZ = dropped_min[dropped_min['Type'].isin([c + '_AMB' for c in npz] + [c + '_UAMB' for c in npz])]

filler_model_syn = smf.mixedlm(
    syntactic_model,
    data=dropped_min_NPZ,
    groups=dropped_min_NPZ["item"],  
    re_formula="1 + syn_surprisal_s + syn_surprisal_p1_s + syn_surprisal_p2_s"
).fit(method="lbfgs", maxiter=100000, disp=True) 

dump(filler_model_syn, model_base + "model-NPZ_onlyGP.joblib")

In [ ]:
dropped_min_only_NPZ = dropped_min[dropped_min['Type'].isin(['NPZ_UAMB', 'NPZ_AMB'])]


predictRT_dataset(syn_model, 
                  os.path.join(model_base, "model-NPZ_onlyGP.joblib"),
                  dropped_min_only_NPZ, 
                  os.path.join(prediction_base, "NPZ_pred_onlyGP.csv"))

##### Inspecting Models

In [8]:
inspect_model(os.path.join(model_base, "model-MVRR_onlyGP.joblib"))
inspect_model(os.path.join(model_base, "model-NPS_onlyGP.joblib"))
inspect_model(os.path.join(model_base, "model-NPZ_onlyGP.joblib"))

Model at /Users/andrewperun/Desktop/LING_380/Final_Proj/Trial2/regressions/model-MVRR_onlyGP.joblib:
Coefficients:
 Intercept                                      421.135621
syn_surprisal_s                                 15.859941
syn_surprisal_p1_s                              13.757483
syn_surprisal_p2_s                               1.265022
scale(WordPosition)                              1.559555
logfreq_s                                      -21.896016
length_s                                        15.666922
logfreq_s:length_s                               0.387973
logfreq_p1_s                                   -13.322270
length_p1_s                                      3.437683
logfreq_p1_s:length_p1_s                        -5.453208
logfreq_p2_s                                   -12.884434
length_p2_s                                     -7.777838
logfreq_p2_s:length_p2_s                       -11.130806
Group Var                                        0.001784
Group x syn_su

##### Getting GP effect from new RT predictions

In [259]:
MVRR_preds_2 = split_predictions(os.path.join(prediction_base, "MVRR_pred_onlyGP.csv"))
NPS_preds_2  = split_predictions(os.path.join(prediction_base, "NPS_pred_onlyGP.csv"))
NPZ_preds_2  = split_predictions(os.path.join(prediction_base, "NPZ_pred_onlyGP.csv"))

In [260]:
MVRR_human_GP_2, MVRR_model_GPs_2 = get_GPeffect_byModel(MVRR_preds_2, lagged_sep, 'mvrr')
NPS_human_GP_2,  NPS_model_GPs_2  = get_GPeffect_byModel(NPS_preds_2,  lagged_sep, 'nps')
NPZ_human_GP_2,  NPZ_model_GPs_2  = get_GPeffect_byModel(NPZ_preds_2,  lagged_sep, 'npz')


In [261]:
# Should be same as human GP in experiment 1
print(MVRR_human_GP_2)
print(NPS_human_GP_2)
print(NPZ_human_GP_2)

[-7.5711756860257005, 41.737653392294625, 167.4356757427677, 94.73673452609182]
[17.100729397502587, 32.337040211112644, 49.61063704069727, 18.32235934651217]
[5.757101983650362, 84.22053326940727, 119.79536401912738, 51.919479000115764]


In [262]:
print(MVRR_model_GPs_2)
print(NPS_model_GPs_2)
print(NPZ_model_GPs_2)

[[-9.339772367011326, 20.10839685793695, 27.018904114778778, 11.247445301578226], [-7.820421732195722, 14.775854930736758, 20.509725451776347, 8.29096741024082], [-5.335067435438879, 18.25203295549088, 20.770113294755795, 7.121386093520472], [-15.803930798748292, 24.266202118193117, 34.32271181381055, 13.14833135207386]]
[[9.67057455685505, 16.42639785666187, 22.85179183211333, 11.216716922543617], [7.45254369961437, 17.103169266332824, 26.54465374534607, 13.84108675351633], [7.63433747870949, 19.086659276201924, 23.943154219220407, 8.263084374563816], [7.647498575587257, 12.743781432423235, 19.700115936424215, 8.505523838181348]]
[[-3.676067337020094, 20.150581982951255, 27.046147552583204, 15.14101262643694], [-2.9939768998268357, 17.284214846857008, 24.90060068653592, 12.951612196823186], [-1.4802614735945676, 19.168520552500073, 22.353437304213475, 9.692395835927428], [1.4432907334748295, 20.982119660165854, 27.80828441714456, 13.560491733887142]]


In [18]:
full_list_2 = [MVRR_model_GPs_2, NPS_model_GPs_2, NPZ_model_GPs_2]
gpath_cases = ['MVRR', 'NPS', 'NPZ']

In [20]:
for i in range(len(full_list_2)):
    data = {
        "m0": full_list_2[i][0],
        "m1": full_list_2[i][1],
        "m2": full_list_2[i][2],
        "m3": full_list_2[i][3]
    }
    df = pd.DataFrame(data)
    title = gpath_cases[i] + "_GPs_2.csv"
    path = os.path.join(base, title)
    df.to_csv(path, index=False)
    

## Prediction of RT & GP Effect of in-sample data (Experiment 2)

In [32]:
def split_predictions_2(preds_path):

    df = pd.read_csv(preds_path)
    split = {}

    # iter through construction type
    for col_2_value in np.unique(df.iloc[:, 2]): 
        filtered_col_2 = df[df.iloc[:, 2] == col_2_value]
        split[col_2_value] = {}

        #  iter through model type
        for col_18_value in np.unique(filtered_col_2.iloc[:, 18]):
            filtered_col_18 = filtered_col_2[filtered_col_2.iloc[:, 18] == col_18_value]
            split[col_2_value][col_18_value] = {}

            # iter through item number (sample sentence ID)
            for col_17_value in np.unique(filtered_col_18.iloc[:, 17]):
                filtered_col_17 = filtered_col_18[filtered_col_18.iloc[:, 17] == col_17_value]
                split[col_2_value][col_18_value][col_17_value] = filtered_col_17

    return split

In [58]:
def get_GPeffect_inSample(predictions, lagged_truth, const):

    GP_effects = []
    for i in predictions[const.upper() + '_UAMB']['m0'].keys(): # for each sentence
        
        i_list = []
        
        # get word position of disambiguating word
        truth = lagged_truth[i][const][const.upper() + '_UAMB']['m0']
        ambig_disambig = truth['disambPositionAmb'].iloc[0]
        unambig_disambig = truth['disambPositionUnamb'].iloc[0]
        #print(f"Item {i}: {ambig_disambig}")
        #print(f"Item {i}: {unambig_disambig}")



        for model in ['m0', 'm1', 'm2', 'm3']:

            m_list = []

            for j in [-1, 0, 1, 2]:
                
                unamb =  predictions[const.upper() + '_UAMB'][model][i]#.iloc[:,9] # 9 indexes word position
                amb  =  predictions[const.upper() + '_AMB'][model][i]#.iloc[:,9] 

                df1 = unamb[unamb.iloc[:,9] == j + unambig_disambig].iloc[:,21] # 21 column indexes new predicted RT
                df2 = amb[amb.iloc[:,9]  == j + ambig_disambig].iloc[:,21]
                #min_len = min([len(df1), len(df2)])

                df1 = df1.reset_index(drop=True)
                df2 = df2.reset_index(drop=True)

                pred_unamb_rts = df1[:].apply(lambda x: float(x[-25:-18])).mean()
                
                pred_amb_rts   = df2[:].apply(lambda x: float(x[-25:-18])).mean()
                #print("Uamb: " + str(pred_unamb_rts))
                #print("Amb: " + str(pred_amb_rts))
                
                diffs = (pred_amb_rts-pred_unamb_rts).tolist()
                #print(diffs)
                m_list.append(np.mean(diffs))

            i_list.append(m_list)
            
        GP_effects.append(i_list)

    #print(GP_effects)
    trans = list(zip(*GP_effects))
    means = []
    for group in trans:
        mean_sublist = np.mean(group, axis=0)
        means.append(mean_sublist)

    model_GP_effects = [list(sublist) for sublist in means]
    
    return model_GP_effects

In [51]:
def get_avg_save(GP_effect_list, out_path, col1, col2):
    
    averages_A = [sum(x) / len(x) for x in zip(*GP_effect_list[0])]
    averages_B = [sum(x) / len(x) for x in zip(*GP_effect_list[1])]
    
    df = pd.DataFrame({col1: averages_A, col2: averages_B})
    df.to_csv(out_path, index=False)

In [40]:
dropped_min = pd.read_csv(out_GP, header=0)

In [41]:
dropped_min_MVRR_NPS = dropped_min[dropped_min['Type'].isin(['MVRR_AMB', 'NPS_AMB', 'MVRR_UAMB', 'NPS_UAMB'])]
dropped_min_MVRR_NPZ = dropped_min[dropped_min['Type'].isin(['MVRR_AMB', 'NPZ_AMB', 'MVRR_UAMB', 'NPZ_UAMB'])]
dropped_min_NPS_NPZ  = dropped_min[dropped_min['Type'].isin(['NPS_AMB', 'NPZ_AMB', 'NPS_UAMB', 'NPZ_UAMB'])]

In [43]:
syn_model = """
    RT ~ syn_surprisal_s + syn_surprisal_p1_s +
         syn_surprisal_p2_s + scale(WordPosition) +
         logfreq_s * length_s + logfreq_p1_s * length_p1_s +
         logfreq_p2_s * length_p2_s
    """

random_effect = "1 + syn_surprisal_s + syn_surprisal_p1_s + syn_surprisal_p2_s"

In [163]:
predictRT_dataset(syn_model, 
                  os.path.join(model_base, "model-MVRR_onlyGP.joblib"),
                  dropped_min_NPS_NPZ, 
                  os.path.join(prediction_base, "ex2_insample_NPS_NPZ_pred_onlyGP.csv"))

/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_27910/1558022743.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["fe_prediction"] = fe_matrix @ fe_params
/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_27910/1558022743.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["rand_effects"] = dataset["item"].map(rand_effects).fillna(0)
/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_27910/1558022743.py:13: SettingWithCopyWarning: 
A value is trying to be set 

In [164]:
predictRT_dataset(syn_model, 
                  os.path.join(model_base, "model-NPS_onlyGP.joblib"),
                  dropped_min_MVRR_NPZ, 
                  os.path.join(prediction_base, "ex2_insample_MVRR_NPZ_pred_onlyGP.csv"))

/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_27910/1558022743.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["fe_prediction"] = fe_matrix @ fe_params
/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_27910/1558022743.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["rand_effects"] = dataset["item"].map(rand_effects).fillna(0)
/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_27910/1558022743.py:13: SettingWithCopyWarning: 
A value is trying to be set 

In [44]:
predictRT_dataset(syn_model, 
                  os.path.join(model_base, "model-NPZ_onlyGP.joblib"),
                  dropped_min_MVRR_NPS,
                  os.path.join(prediction_base, "ex2_insample_MVRR_NPS_pred_onlyGP.csv"))

/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_55479/1558022743.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["fe_prediction"] = fe_matrix @ fe_params
/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_55479/1558022743.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["rand_effects"] = dataset["item"].map(rand_effects).fillna(0)
/var/folders/cp/13y3b68j0tz9h3vtqsf6lb400000gn/T/ipykernel_55479/1558022743.py:13: SettingWithCopyWarning: 
A value is trying to be set 

In [45]:
sep_preds_NPS_NPZ  = split_predictions_2(os.path.join(prediction_base, "ex2_insample_NPS_NPZ_pred_onlyGP.csv"))
sep_preds_MVRR_NPZ = split_predictions_2(os.path.join(prediction_base, "ex2_insample_MVRR_NPZ_pred_onlyGP.csv"))
sep_preds_MVRR_NPS = split_predictions_2(os.path.join(prediction_base, "ex2_insample_MVRR_NPS_pred_onlyGP.csv"))

In [17]:
columns_to_check = [
        'lex_surprisal_s', 'lex_surprisal_p1_s', 'lex_surprisal_p2_s', 'lex_surprisal_p3_s',
        'syn_surprisal_s', 'syn_surprisal_p1_s', 'syn_surprisal_p2_s', 'syn_surprisal_p3_s',
        'logfreq_s', 'logfreq_p1_s', 'logfreq_p2_s', 'logfreq_p3_s']

lagged_sep = split_Lags(os.path.join(base, "with_lags.csv"), columns_to_check)

In [59]:
GP_effect_MVRR_1 = get_GPeffect_inSample(sep_preds_MVRR_NPZ, lagged_sep, 'mvrr') # model A
GP_effect_MVRR_2 = get_GPeffect_inSample(sep_preds_MVRR_NPS, lagged_sep, 'mvrr') # model B

In [60]:
GP_effect_NPS_1 = get_GPeffect_inSample(sep_preds_MVRR_NPS, lagged_sep, 'nps') # model B
GP_effect_NPS_2 = get_GPeffect_inSample(sep_preds_NPS_NPZ, lagged_sep, 'nps') # model C

In [61]:
GP_effect_NPZ_1 = get_GPeffect_inSample(sep_preds_MVRR_NPZ, lagged_sep, 'npz') # model A
GP_effect_NPZ_2 = get_GPeffect_inSample(sep_preds_NPS_NPZ, lagged_sep, 'npz') # model C

In [62]:
MVRR_out = os.path.join(base, "MVRR_control_2.csv")
NPS_out =  os.path.join(base, "NPS_control_2.csv")
NPZ_out =  os.path.join(base, "NPZ_control_2.csv")

In [63]:
get_avg_save([GP_effect_MVRR_1, GP_effect_MVRR_2], MVRR_out, 'A', 'B')
get_avg_save([GP_effect_NPS_1, GP_effect_NPS_2], NPS_out, 'B', 'C')
get_avg_save([GP_effect_NPZ_1, GP_effect_NPZ_2], NPZ_out, 'A', 'C')
